In [ ]:
"""
Function that generates pd.DataFrame with relative and absolute noise level

Parameters: 

full_df: Dataframe
Dataframe with fourier spectrum

peak_threshold: float, from 0 to 20, default is 20
number of peaks above threshold

Returns: 

features: DataFrame, shape=(full_df[1]/2, 1)
number of peaks
"""

def noise_level(full_df, peak_threshold=20):
    all_props = []
    df = full_df[full_df.columns[::2]]
    for col in df:
        _, properties = find_peaks(df[col], height=0)
        all_props.append(properties)
    
    noise_levels_rel=[]
    noise_levels_abs=[]
    
    for i in range(len(all_props)):
        peak_list=[]
        max_height = np.max(all_props[i]['peak_heights'])
        
        for j in range(0,1000):
            peaks, _ = find_peaks(df.iloc[:,i], threshold = (j/1000)*max_height)
            peak_list.append(len(peaks))
        try:    
            noise_level_rel = peak_list.index(peak_threshold)/1000
        except ValueError:
            try:
                noise_level_rel = peak_list.index(peak_threshold+1)/1000
            except ValueError:
                noise_level_rel = 0
                print(i)
        noise_level_abs = noise_level_rel * max_height 
        noise_levels_rel.append(noise_level_rel)
        noise_levels_abs.append(noise_level_abs)
    noise_level_rel = pd.DataFrame(noise_levels_rel, columns=['noise_level_rel'])
    noise_level_abs = pd.DataFrame(noise_levels_abs, columns=['noise_level_abs'])
    
    noise_level = pd.concat([noise_level_rel, noise_level_abs], axis=1)
    return(noise_level)

In [ ]:
"""
Function that generates pd.DataFrame with different properties for n highest peaks. Properties - values of frequency, 
height, width of the peak for half peak height, prominence and existence of second harmonics. 

In more details about properties https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html

Parameters: 

full_df: Dataframe
Dataframe with fourier spectrum

n: int
number of peaks

lf_th: float
threshold for low frequency noise (1Hz by default)

Returns: 

features: DataFrame, shape=(full_df.shape[1]/2, n*4)
properties of n hightest peaks
"""

def properties_of_peaks(full_df, n, lf_th=1):
    
    all_peaks = []
    all_props = []
    df = full_df[full_df.columns[::2]][:-1]
    xf = full_df[full_df.columns[1::2]][:-1]
    
    for col in df:
        peaks, properties = find_peaks(df[col], height=0, width=0, prominence=0, rel_height=0.5)
        properties['peak_index'] = peaks
        all_props.append(properties)
        all_peaks.append(peaks)
        
    
    index = []
    freq = []
    height = []
    width = []
    prominence = []
    low_noise = []
    freq_forharm =[] # list for second harmonics detection (binary classification)
    
    features = pd.DataFrame()
    
    for i in range(len(all_props)):
        z = np.argsort(all_props[i]['peak_heights'])
        z = z[:-(n+1):-1]
        idx = all_props[i]['peak_index'][z] #index of max peaks
        
        fr_forharm = xf.iloc[:,i][all_peaks[i][z]] # list of frequencies for max peaks
        index_forharm = 0 #initiation of second harmonics index (for each i) 
        for q in range(n):
            for p in range(n):
                try:
                    a=fr_forharm[p]/fr_forharm[q] #frequencies relation
#                 print(i,a)
                    if (a<2.1) and (a>1.9): index_forharm = 1 # if relation is 2 plus/minus 5% output 1 
                except ZeroDivisionError:
                    a=0 
        freq_forharm.append(index_forharm)
        
        for j in range(n):
            fr = xf.iloc[:,i][all_peaks[i][z][j]] #freqs of max peaks
            freq.append(fr)
            
        h = all_props[i]['peak_heights'][z] #heights of max peaks
        height.append(h)
        w = all_props[i]['widths'][z] #width of max peaks
        width.append(w)
        p = all_props[i]['prominences'][z]
        prominence.append(p)

        
    freq = np.reshape(freq, (len(all_props), n)) 
    for i in range(freq.shape[0]):
        cond = np.where(freq[i] < lf_th)
#         print(cond[0])
#         print(cond[0].size)
        if cond[0].size == 0:
            low_noise.append(0)
        else:
            low_noise.append(1)
            
            
    freq = pd.DataFrame(freq, columns=['freq ' + str(i) for i in range(n)])
    height = pd.DataFrame(height, columns=['height ' + str(i) for i in range(n)])
    width = pd.DataFrame(width, columns=['width ' + str(i) for i in range(n)])
    prominence = pd.DataFrame(prominence, columns=['prominence ' + str(i) for i in range(n)])
    low_noise = pd.DataFrame(low_noise, columns=['low_freq_noise'])
    freq_forharm = pd.DataFrame(freq_forharm, columns=['second harmonics '])
    
    features = pd.concat([features, freq, height, width, prominence, low_noise, freq_forharm], axis=1)
    
    return(features)